In [ ]:
from utils import *

In [ ]:
from azureml.core import Workspace

# Configure experiment
ws = Workspace.from_config()

In [ ]:
# Create or get training cluster
aml_cluster = get_aml_cluster(ws, cluster_name="cpu-cluster")
aml_cluster.wait_for_completion(show_output=True)

In [ ]:
# Create a run configuration
run_conf = get_run_config(['numpy', 'pandas', 'scikit-learn', 'tensorflow'])

In [ ]:
from azureml.core import Dataset

dataset = Dataset.get_by_name(ws, name='titanic')
data_in = dataset.as_named_input('titanic')

In [ ]:
from azureml.core import Datastore
from azureml.pipeline.core import PipelineData

datastore = Datastore.get(ws, datastore_name="mldata")
data_train = PipelineData('train', datastore=datastore)
data_test = PipelineData('test', datastore=datastore)

In [ ]:
from azureml.data import OutputFileDatasetConfig

data_out = OutputFileDatasetConfig(name="predictions", destination=(datastore, 'titanic/predictions'))
data_out = data_out.read_delimited_files().register_on_complete('titanic.pred')

In [ ]:
from azureml.pipeline.steps import PythonScriptStep

step_1 = PythonScriptStep(name='Preprocessing',
                          script_name="preprocess_output.py",
                          source_directory="code",
                          arguments=[
                              "--input", data_in,
                              "--out-train", data_train,
                              "--out-test", data_test],
                          inputs=[data_in],
                          outputs=[data_train, data_test],
                          runconfig=run_conf,
                          compute_target=aml_cluster)


In [ ]:
from azureml.pipeline.core.graph import PipelineParameter

lr_param = PipelineParameter(name="lr_arg", default_value=0.01)

In [ ]:
from azureml.pipeline.steps import PythonScriptStep

step_2 = PythonScriptStep(name='Training',
                          script_name="train_output.py",
                          source_directory="code",
                          arguments=[
                              "--in-train", data_train,
                              "--in-test", data_test,
                              "--output", data_out,
                              "--learning-rate", lr_param],
                          inputs=[data_train, data_test],
                          outputs=[data_out],
                          runconfig=run_conf,
                          compute_target=aml_cluster)

In [ ]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(ws, steps=[step_1, step_2])

In [ ]:
pipeline.validate()

In [ ]:
service = pipeline.publish(name="AzureML Published Pipeline", version="1.0")

print(service)
service_id = service.id
service_endpoint = service.endpoint

In [ ]:
from azureml.pipeline.core import PipelineEndpoint

endpoint = PipelineEndpoint.publish(ws, pipeline=service, name="AzureML Published Pipeline Endpoint", description="Mastering Azure Machine Learning")

print(endpoint)
service_id = endpoint.id
service_endpoint = endpoint.endpoint

In [ ]:
from azureml.core.authentication import AzureCliAuthentication

cli_auth = AzureCliAuthentication()
aad_token = cli_auth.get_authentication_header()

In [ ]:
import requests

request = {
    "ExperimentName": "azureml-pipeline-trigger",
    "ParameterAssignments": {
        "lr_arg": 0.05
    }
}

response = requests.post(service_endpoint, headers=aad_token, json=request)
print(response.json)

In [ ]:
# service.disable()
# endpoint.disable()